In [ ]:
%pip install shareplum
!pip show shareplum

### Descarga "aenc" y "tfroc" desde el FTP y carga en el SharePoint - Mes actual y mes anterior

In [1]:
from ftplib import FTP_TLS
from shareplum import Site
from shareplum import Office365
from shareplum.site import Version
import os
from datetime import datetime, timedelta
import requests

# Aumentar el tiempo de espera global para las solicitudes HTTP
requests.adapters.DEFAULT_RETRIES = 5
requests.adapters.DEFAULT_TIMEOUT = 30

# 🔹 DATOS DE CONEXIÓN FTP
ftp_server = "xmftps.xm.com.co"
ftp_port = 210
ftp_user = "ISAMDNT\\1098742265"
ftp_password = "Ru1t0qu309p2026."

# 🔹 DATOS DE SHAREPOINT
sharepoint_url = "https://ruitoqueesp1.sharepoint.com"
sharepoint_site = "fronterascomerciales"  # Nombre del sitio en SharePoint
sharepoint_user = "rbadillo@ruitoqueesp.com"
sharepoint_password = "r2083502R"

# 🔹 VARIABLES PARA AÑO Y MES (MODIFICABLES)
anio_especifico = None  # Cambia esto si quieres un año específico (ej: 2023)
mes_especifico = None   # Cambia esto si quieres un mes específico (ej: 2 para febrero)

# 🔹 OBTENER AÑO Y MES SEGÚN CONFIGURACIÓN
def obtener_fecha(anio_especifico=None, mes_especifico=None):
    if anio_especifico and mes_especifico:
        return anio_especifico, f"{mes_especifico:02d}"
    else:
        anio_actual = datetime.now().year
        numero_mes = datetime.now().strftime("%m")
        return anio_actual, numero_mes

# 🔹 FUNCIONES AUXILIARES
def obtener_mes_anterior(anio, mes):
    fecha_actual = datetime(anio, int(mes), 1)
    primer_dia_mes_actual = fecha_actual.replace(day=1)
    ultimo_dia_mes_anterior = primer_dia_mes_actual - timedelta(days=1)
    return ultimo_dia_mes_anterior.year, f"{ultimo_dia_mes_anterior.month:02d}"

def filtrar_archivos(version, prefijo):
    prioridad_versiones = [".TxF", ".TxR", ".Tx2"]
    for extension in prioridad_versiones:
        archivos_filtrados = [archivo for archivo in version if archivo.startswith(prefijo) and archivo.endswith(extension)]
        if archivos_filtrados:
            return archivos_filtrados
    return []

def verificar_archivos_sharepoint(folder, prefijo):
    archivos_sharepoint = folder.files
    archivos_nombres = [archivo['Name'] for archivo in archivos_sharepoint]
    archivos_txF = [archivo for archivo in archivos_nombres if archivo.startswith(prefijo) and archivo.endswith(".TxF")]
    return archivos_txF

def limpiar_archivos_sharepoint(folder, archivos):
    """
    Limpia los archivos en SharePoint según la jerarquía TxF > TxR > Tx2
    """
    # Obtener todos los archivos en el folder
    archivos_sharepoint = folder.files
    archivos_nombres = [archivo['Name'] for archivo in archivos_sharepoint]
    
    # Separar archivos por extensión
    archivos_txf = [archivo for archivo in archivos_nombres if archivo.endswith(".TxF")]
    archivos_txr = [archivo for archivo in archivos_nombres if archivo.endswith(".TxR")]
    archivos_tx2 = [archivo for archivo in archivos_nombres if archivo.endswith(".Tx2")]
    
    archivos_borrar = []
    
    # Si existen archivos TxF, borrar TxR y Tx2
    if archivos_txf:
        archivos_borrar.extend(archivos_txr + archivos_tx2)
    # Si no hay TxF pero hay TxR, borrar solo Tx2
    elif archivos_txr:
        archivos_borrar.extend(archivos_tx2)
    
    # Borrar los archivos seleccionados
    for archivo in archivos_borrar:
        try:
            folder.delete_file(archivo)
            print(f"Archivo eliminado de SharePoint: {archivo}")
        except Exception as e:
            print(f"Error al eliminar {archivo}: {e}")

def procesar_mes(anio, mes):
    ruta_ftp = f"/INFORMACION_XM/USUARIOSK/RTQC/sic/comercia/{anio}-{mes}"
    sharepoint_folder = f"Documentos Compartidos/aenc/{anio}/{mes}"

    # 🔹 CONECTARSE A SHAREPOINT Y VERIFICAR EXISTENCIA DE ARCHIVOS
    try:
        print("Conectando a SharePoint...")
        authcookie = Office365(sharepoint_url, username=sharepoint_user, password=sharepoint_password).GetCookies()
        site = Site(f"{sharepoint_url}/sites/{sharepoint_site}", authcookie=authcookie, version=Version.v365)
        folder = site.Folder(sharepoint_folder)
        print("Conexión a SharePoint establecida.")

        # Verificar si ya existen archivos con extensión ".TxF" en SharePoint
        archivos_txF = verificar_archivos_sharepoint(folder, "aenc")
        if archivos_txF:
            print(f"Archivos con extensión '.TxF' ya existen en SharePoint para {anio}-{mes}. No se descargará nada.")
            return
        else:
            print(f"No se encontraron archivos con extensión '.TxF' en SharePoint para {anio}-{mes}. Procediendo con la descarga.")

    except Exception as e:
        print(f"Error en la conexión a SharePoint: {e}")
        return

    # Cerrar la conexión a SharePoint
    del site

    # 🔹 CONECTARSE A FTP
    try:
        print(f"Conectando al servidor FTP para {anio}-{mes}...")
        connection = FTP_TLS()
        connection.connect(ftp_server, ftp_port)
        connection.login(user=ftp_user, passwd=ftp_password)
        connection.set_pasv(True)
        connection.prot_p()
        print("Conexión establecida.")

        # Navegar a la carpeta en el FTP
        connection.cwd(ruta_ftp)
        print(f"Navegando a la ruta FTP: {ruta_ftp}")

        # Listar archivos y filtrar los que comienzan con "aenc" o "tfroc"
        archivos_ftp = connection.nlst()

        # Filtrar archivos por prioridad para "aenc" y "tfroc"
        archivos_aenc = filtrar_archivos(archivos_ftp, "aenc")
        archivos_tfroc = filtrar_archivos(archivos_ftp, "tfroc")

        # Combinar las listas de archivos a descargar
        archivos_descargar = archivos_aenc + archivos_tfroc

        if not archivos_descargar:
            print("No se encontraron archivos para descargar.")
        else:
            print(f"Archivos a descargar: {archivos_descargar}")

            # Crear carpeta temporal para almacenar archivos descargados
            temp_folder = "archivos_descargados"
            os.makedirs(temp_folder, exist_ok=True)

            for archivo in archivos_descargar:
                local_path = os.path.join(temp_folder, archivo)
                with open(local_path, "wb") as f:
                    connection.retrbinary(f"RETR {archivo}", f.write)
                print(f"Archivo descargado: {archivo}")

    except Exception as e:
        print(f"Error en la conexión FTP: {e}")

    finally:
        if 'connection' in locals():
            connection.quit()
            print("Conexión FTP cerrada.")

    # 🔹 SUBIR ARCHIVOS A SHAREPOINT
    try:
        print("Conectando a SharePoint para subir archivos...")
        authcookie = Office365(sharepoint_url, username=sharepoint_user, password=sharepoint_password).GetCookies()
        site = Site(f"{sharepoint_url}/sites/{sharepoint_site}", authcookie=authcookie, version=Version.v365)
        folder = site.Folder(sharepoint_folder)
        print("Conexión a SharePoint establecida.")

        # Subir los archivos a SharePoint
        for archivo in archivos_descargar:
            local_path = os.path.join(temp_folder, archivo)
            with open(local_path, "rb") as file:
                file_content = file.read()
                folder.upload_file(file_content, archivo)
                print(f"Archivo subido a SharePoint: {archivo}")
        
        # Limpiar archivos según la jerarquía después de subir
        limpiar_archivos_sharepoint(folder, archivos_descargar)

    except Exception as e:
        print(f"Error en la subida a SharePoint: {e}")

    finally:
        del site

# 🔹 PROCESAR MES ANTERIOR Y MES ACTUAL
anio_actual, mes_actual = obtener_fecha(anio_especifico, mes_especifico)
anio_anterior, mes_anterior = obtener_mes_anterior(anio_actual, mes_actual)

procesar_mes(anio_anterior, mes_anterior)
procesar_mes(anio_actual, mes_actual)

# Mensaje final
print("Proceso finalizado")

Conectando a SharePoint...
Error en la conexión a SharePoint: Shareplum HTTP Post Failed : 403 Client Error: Forbidden for url: https://ruitoqueesp1.sharepoint.com/_forms/default.aspx?wa=wsignin1.0
Conectando a SharePoint...
Error en la conexión a SharePoint: Shareplum HTTP Post Failed : 403 Client Error: Forbidden for url: https://ruitoqueesp1.sharepoint.com/_forms/default.aspx?wa=wsignin1.0
Proceso finalizado


### Borrar el contenido de la carpeta `archivos_descargados`

In [2]:
import os
import shutil

temp_folder = "archivos_descargados"

# Verificar si la carpeta existe
if os.path.exists(temp_folder):
    # Borrar el contenido de la carpeta
    for filename in os.listdir(temp_folder):
        file_path = os.path.join(temp_folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print(f'No se pudo borrar {file_path}. Razón: {e}')
    print(f"El contenido de la carpeta '{temp_folder}' ha sido borrado.")
else:
    print(f"La carpeta '{temp_folder}' no existe.")

El contenido de la carpeta 'archivos_descargados' ha sido borrado.
